# Covid-19 Risk Factor Predictor


## Our Contribution
**We predict important risk factors for COVID-19 and we rank them by importance**. Risk factors can be characteristics, conditions, or behaviours. 

We build a **knowledge graph** by merging  entities and relations that we extract from the **CORD-19 corpus** with the **[Open Targets Genetics](https://genetics-docs.opentargets.org/)** dataset and the [Mondo Disease Ontology](https://www.ebi.ac.uk/ols/ontologies/mondo). This is done to build a graph that merges COVID-19 facts extracted from bleeding-edge literature and known facts on similar diseases stored in datasets online. We achieve such goal despite the absence of COVID-19 from OpenTargets.

Loosely inspired by [Mendelian Randomization](https://en.wikipedia.org/wiki/Mendelian_randomization), we train a **[graph representation learning model](https://github.com/Accenture/AmpliGraph)** that learns vector representations of the concepts in the graph (i.e. **graph embeddings**). Such operation leverages the topology and the semantics of the graph to discover new associations between risk factors and COVID-19. To sanity-check the trained embeddings, we carve out known risk factors-disease associations for better-studied conditions such as SARS, MERS, and we assess the predictive power of the model using agreed-upon evaluation protocol and metrics. Our model reaches a validation mean reciprocal rank MRR=0.07.

Finally, we use the knowledge graph embeddings to infer how likely COVID-19 is impacted by a risk factor and we return a **ranked list of predicted risk factors** for COVID-19.
Our results can be used either to test if a suspected risk factor applies to COVID-19, or to discover unsuspected risk factors.

**The complete pipeline of the system is presented below**:

![pipeline](https://kagglecovid.s3-eu-west-1.amazonaws.com/covid19_pipeline.jpg)


### Benefits
* First machine learning system that returns a ranked list of predited risk factors for COVID-19.
* The knowledge graph can easily be extended and updated with additional information (e.g. paper citation network, additional genomic datasets, etc.).
* Graph representation learning exploits relations between concepts shared across articles, and it is scalable to support larger graphs (up to 500+M edges).

### Known Limitations
* In this Phase 1 prototype, we limited to CORD-19 **biorxiv articles** only.
* Our approach combining paper annotation with Open Targets assumes genetics and genomics play a role in the impact on COVID-19.
* The NER step likely missed some entities in the text and mis-identified others. Our second step mapping these entities identified as keyword into entities identified with unique identifier introduces a second level of possible mapping errors;
* The risk factor surfaced are correlations emerging from the structure of the graph, not causations. Our graph now an association network associating papers to the things they are about, and associating these things to each other;
* Due to time constraints, we did not carry out exhaustive model selection (hyperparameter tuning), to the detriment of predictive power.
* The embeddings have to be re-trained every time the network is changed (e.g. when new papers are added). This is a known limitation of knowledge graph embedding models.

### Future Work Planned for Phase 2
* Re-build the graph on all CORD-19 papers
* Validation of the final output by domain experts.
* For each COVID-19 - Risk Factor prediction, we will also provide an explanation, in the form of a meaningful and succint subgraph that mostly affected such prediction.
* Extensive and thorough hyperparameter tuning to improve predictive power.
* The machine learning model will be adapted to support the addition of new concepts (e.g. newly-published papers), without the need to re-train the model.
* The work described in this notebook will be part of a [wider pipeline](https://www.kaggle.com/cgueret/covid-19-risk-factor-predictor-future-work) aimed at discovering additional COVID-19 insights from the CORD-19 corpus. 



### Results

**Scroll down at the end of the notebook to see the list of COVID-19 risk factors predicted by our system.**




# Part 1/3: NER

The first component of our system is the named-entity recognition & relation extraction.
It reads the CORD-19 corpus and extracts entities and relations that will be used later on to generate the knowledge graph.

![](https://kagglecovid.s3-eu-west-1.amazonaws.com/covid19_pipeline_1.jpg)

**The NER pipeline performs the following operations**:
1. Extract concepts from the papers using Spacy and the models from [SciSpacy](https://allenai.github.io/scispacy/).
2. Turn the found keywords into identifiers using a mapping extracted from [Open Targets]((https://genetics-docs.opentargets.org/)) and the [Mondo Disease Ontology](https://www.ebi.ac.uk/ols/ontologies/mondo).

The NER pipeline is reported in the cells below:

In [ ]:
import spacy
spacy.prefer_gpu()
import json
from tqdm.auto import tqdm
from pathlib import Path
import pandas as pd
import numpy as np
import gzip
!pip install opentargets
from opentargets import OpenTargetsClient
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bionlp13cg_md-0.2.4.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_ner_bc5cdr_md-0.2.4.tar.gz

This function looks at all the papers in the CORD-19 dataset and extract entities. 

Note to speed up processing time, we only process articles from biorxiv:

In [0]:
def extract_paper_annotations():
    """
    This function looks at all the papers in the CORD-19 dataset and extract entities
    """
    # Define the list of papers we will process
    #papers = [Path("/kaggle/input/CORD-19-research-challenge/biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/59eab95c43fdea01481fdbf9bae45dfe28ffc693.json")]
    papers = [p for p in Path('/kaggle/input/CORD-19-research-challenge').glob('biorxiv_medrxiv/biorxiv_medrxiv/pdf_json/*.json')]
    #papers += [p for p in Path('/kaggle/input/CORD-19-research-challenge').glob('comm_use_subset/comm_use_subset/pdf_json/*.json')]
    #papers += [p for p in Path('/kaggle/input/CORD-19-research-challenge').glob('noncomm_use_subset/noncomm_use_subset/pdf_json/*.json')]
    #papers += [p for p in Path('/kaggle/input/CORD-19-research-challenge').glob('custom_license/custom_license/pdf_json/*.json')]
    print (len(papers)) 

    # Load the NLP models
    nlp_model_bionlp13cg = spacy.load('/opt/conda/lib/python3.6/site-packages/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.4') # For cells, genes, ...
    nlp_model_bc5cdr = spacy.load('/opt/conda/lib/python3.6/site-packages/en_ner_bc5cdr_md/en_ner_bc5cdr_md-0.2.4') # For diseases

    # The output will be one hashmap associating each paper to its annotations
    output = {}

    # Process all the papers
    for paper in tqdm(papers):
        try:
            # Load the document
            document = json.loads(paper.read_text())

            # Get the ID
            paper_id = document['paper_id']
            
            # Initialise its entry
            output[paper_id] = {}
            output[paper_id]['topics'] = {} # The different topic annotations grouped per type
            
            # Group the text by sections (took more than 9h to process!)
            #section_texts = {}
            #section_texts['abstract'] = []
            #for b in document['abstract']:
            #    section_texts['abstract'].append(b['text'])
            #for b in document['body_text']:
            #    section_texts.setdefault(b['section'], [])
            #    section_texts[b['section']].append(b['text'])

            # Retrieve all the text
            texts = []
            for b in document['abstract']:
                texts.append(b['text'])
            if 'body_text' in document:
                for b in document['body_text']:
                    texts.append(b['text'])
            
            # Process the different sections to extract entities
            #for section,texts in section_texts.items():
            text = '.'.join(texts)
            for nlp_model in [nlp_model_bionlp13cg, nlp_model_bc5cdr]:
                tokens = nlp_model(text)
                for entity in tokens.ents:
                    topic_type = entity.label_
                    topic_value = str(entity.text)
                    output[paper_id]['topics'].setdefault(topic_type, set())
                    output[paper_id]['topics'][topic_type].add(topic_value)
            
        except Exception as e:
            print ('Error with {}'.format(paper))
            print (e)

    # Turn the sets into lists to save them as JSON
    for paper_id in output.keys():
        for topic_type in output[paper_id]['topics'].keys():
            output[paper_id]['topics'][topic_type] = list(output[paper_id]['topics'][topic_type])

    return output

In [0]:
# Step 1 => get the keywords out of the paper abstract and content
annotations = extract_paper_annotations()
print (len(annotations.keys()))

# Part 2/3: Knowledge Graph Generation

In this section we generate the knowledge graph (KG) that part 3/3 will use to predict COVID-19 risk factors associations.

![](https://kagglecovid.s3-eu-west-1.amazonaws.com/covid19_pipeline_2.jpg)

### Knowledge Graph Schema
The knowledge graph models relations between CORD-19 papers (extracted by part 1/3) and Genes, Diseases retrieved from [Open Targets Genetics](https://genetics-docs.opentargets.org/).
This is the schema of the knowledge graph that we generate to predict covid19-specific risk factors:

![](https://kagglecovid.s3-eu-west-1.amazonaws.com/covid_kg_ontology_v6_resized.png)

+ **Paper**: an article from the CORDI-19 corpus.
+ **Target**: a gene that occurs in at least one CORD-19 paper, and that may be associated to a Disease.
+ **Disease**: following OpenTargets terminology, this class represents characteristics, conditions, and behaviours. The risk factors predicted by our system belowngs here.

+ **isAboutGene**: connects a CORDI-19 paper to a target gene.
+ **isAboutDisease**: connects a CORDI-19 paper to a Disease (i.e. a characteristic, condition, or behaviour - following OpenTargets terminology).
+ **isASsociatedTo**: connects a target gene to a Disease (i.e. a characteristic, condition, or behaviour - following OpenTargets terminology).
+ **belongsToTherapeuticArea**: associates a Disease to its therapeutic area, as provided by Open Targets.
+ **isASpecific**: this predicate associates a Disease to an higher-up element in the Open Targets hierarchy (ontological path connecting higher classes of diseases to more specific instances).
+ **hasGeneticClue**: this predicate connects a Disease to a characteristic, condition, or behaviour if there are genetic evidences that such connection exists. **As indicated in the picture and the example below, we predict facts that include this predicate and COVID-19 as subject**.

The figure below shows some facts included in the knowledge graph. The figure includes `COVID-19` and the relations that must be predicted in part 3/3.

![covid_Abox.jpg](https://kagglecovid.s3-eu-west-1.amazonaws.com/covid_Abox.jpg)

The graph has 327,834 facts (i.e. edges) and 21,025 distinct concepts (i.e. nodes).
The screenshot below had been generated with [Gephi](https://gephi.org/), and gives a sense of the complexity and size of the data structure:

![graph_gephi.jpg](https://kagglecovid.s3-eu-west-1.amazonaws.com/graph_gephi.jpg)

### Knowledge Graph Construction
The cells below include the code required to generate the knowledge graph.
The most important steps are:
1. We use the entities extracted by Step 1/3.
3. We query Open Targets to enrich the paper annotations with additional information, to connect the topics of the papers to each other (see above for the description of the schema):
  + Link between diseases and genes
  + Link between diseases based on their gene similarity
  + Ontological path connecting higher classes of diseases to more specific instances

This function is used to generate a graph from the paper annotations:

In [0]:
def get_paper_annotations_graph(annotations):
    """
    This function is used to generate a graph from the paper annotations
    
    We will turn all the NLP annotations into concept identifiers using a list of terms extracted form Open Targets and the ontology MONDO. 
    This is done using a basic exact string matching and all the non matching strings are ignored.

    We extract a mapping "disease name" => "disease identifier" from Open Targets as the primary source, falling back on Mondo to fill the gaps. 
    In particular one of the missing value in Open Targets right now is Covid-19 ... ;-)
    """
    
    # Prepare a map to deal with all the different types of entities type recognized by Spacy and that may be found in the annotations
    ontology_map = {
        'DISEASE': {},
        'CANCER': {},
        'GENE_OR_GENE_PRODUCT': {}
    }

    # TODO: If we want to keep more of the annotations returned by Spacy we should align:
    # From https://allenai.github.io/scispacy/ en_ner_bionlp13cg_md
    #  CANCER, ORGAN, TISSUE, ORGANISM, CELL, AMINO_ACID, GENE_OR_GENE_PRODUCT, 
    #  SIMPLE_CHEMICAL, ANATOMICAL_SYSTEM, IMMATERIAL_ANATOMICAL_ENTITY, 
    #  MULTI-TISSUE_STRUCTURE, DEVELOPING_ANATOMICAL_STRUCTURE, 
    #  ORGANISM_SUBDIVISION, CELLULAR_COMPONENT
    # From https://allenai.github.io/scispacy/ en_ner_bc5cdr_md
    #  DISEASE, CHEMICAL

    ########################
    # Get mappings for DISEASE
    ########################
    
    # Load the file from Open Targets and fill the hashmap in
    disease_list = pd.read_csv('https://storage.googleapis.com/open-targets-data-releases/20.02/output/20.02_disease_list.csv.gz', compression='gzip')
    disease_list['disease_full_name'] = disease_list['disease_full_name'].str.lower()
    print('Number of keywords in open targets:', len(set(disease_list['disease_full_name'].values)))
    for index, row in disease_list.iterrows():
        full_name = row['disease_full_name'].lower()
        identifier = row['efo_id']
        ontology_map['DISEASE'][full_name] = identifier

    # Open Targets does not have Covid-19 in its list of diseases. We had it manually
    # To get the labels we ran the following query on http://www.ontobee.org/sparql
    #  select distinct ?s ?o where {
    #    {<http://purl.obolibrary.org/obo/MONDO_0100096> <http://www.geneontology.org/formats/oboInOwl#hasExactSynonym> ?o} 
    #    union
    #    {<http://purl.obolibrary.org/obo/MONDO_0100096> <http://www.w3.org/2000/01/rdf-schema#label> ?o}
    #  }
    ontology_map['DISEASE']['2019 novel coronavirus infection'.lower()] = 'MONDO_0100096'
    ontology_map['DISEASE']['2019-nCoV infection'.lower()] = 'MONDO_0100096'
    ontology_map['DISEASE']['severe acute respiratory syndrome coronavirus 2'.lower()] = 'MONDO_0100096'
    ontology_map['DISEASE']['SARS-CoV-2'.lower()] = 'MONDO_0100096'
    ontology_map['DISEASE']['SARS-coronavirus 2'.lower()] = 'MONDO_0100096'
    ontology_map['DISEASE']['coronavirus disease 2019'.lower()] = 'MONDO_0100096'
    ontology_map['DISEASE']['COVID-19'.lower()] = 'MONDO_0100096'
    
    # Debug output
    print ('Number of keywords in map for diseases: {}'.format(len(ontology_map['DISEASE'])))
    
    
    ########################
    # Get mappings for CANCER
    ########################

    # We will simply treat the "CANCER" annotations from Spacy as "DISEASE"
    ontology_map['CANCER'] = ontology_map['DISEASE']
    
    
    ########################
    # Get mappings for GENE_OR_GENE_PRODUCT
    ########################
    
    # Load a target list from Open Targets. It will be used to map gene keywords
    target_list = pd.read_csv('https://storage.googleapis.com/open-targets-data-releases/20.02/output/20.02_target_list.csv.gz', compression='gzip')
    target_list['hgnc_approved_symbol'] = target_list['hgnc_approved_symbol'].str.lower()
    print('Number of genes in open targets:', target_list['hgnc_approved_symbol'].nunique())
    for index, row in target_list.iterrows():
        full_name = row['hgnc_approved_symbol']
        identifier = row['ensembl_id']
        ontology_map['GENE_OR_GENE_PRODUCT'][full_name] = identifier
    

    ########################
    # Turn the paper annotations into a graph
    ########################
    graph = []
    predicates = {
        'DISEASE': 'isAboutDisease',
        'CANCER': 'isAboutDisease',
        'GENE_OR_GENE_PRODUCT': 'isAboutTarget'
    }
    # Go through all the papers
    for (paper_id, data) in annotations.items():
        # For each annotation topic try to find a match in the ontology map
        for (topic, values) in data['topics'].items():
            if topic in ontology_map:
                for value in values:
                    if value.lower() in ontology_map[topic]:
                        obj = ontology_map[topic][value.lower()]
                        graph.append([paper_id, predicates[topic], obj])
               
    return graph

This function will use the association data from Open Target to 
    connect instances of Target and Disease in the graph.


In [0]:
def connect_targets_and_diseases(graph):
    """
    This function will use the association data from Open Target to 
    connect instances of Target and Disease in the graph.
    
    We at the same time connect diseases to therapeutic areas (instances of Disease)
    as this information is returned by the API
    """
    
    # Get a list of all the targets (genes) and diseases currently in the graph
    targets = list(set([t[2] for t in graph if t[1] == 'isAboutTarget']))
    diseases = list(set([t[2] for t in graph if t[1] == 'isAboutDisease']))
    
    # Prepare a map of target => disease relations
    ot_output_associations = {}
    
    # Query OpenTargets for Target => Disease associations
    ot = OpenTargetsClient()
    for target in tqdm(targets):
        ot_output_associations.setdefault(target, set())
        search_results = ot.get_associations_for_target(target)
        if len(search_results) > 0 and search_results[0]['target']['id'] == target:
            for search_result in search_results:
                if search_result['association_score']['overall'] > 0.8:
                    disease = search_result['disease']['id']
                    ot_output_associations[target].add(disease)
                        
    # Query OpenTargets for Disease => Target associations
    for disease in tqdm(diseases): 
        search_results = ot.get_associations_for_disease(disease)
        if len(search_results) > 0 and search_results[0]['disease']['id'] == disease:
            for search_result in search_results:
                if search_result['association_score']['overall'] > 0.8:
                    target = search_result['target']['id']
                    ot_output_associations.setdefault(target, set())
                    ot_output_associations[target].add(disease)

    # Turn the output into new edges in the graph
    for target, diseases in ot_output_associations.items():
        for disease in diseases:
            # Target -> Disease relation
            graph.append([target, 'isAssociatedTo', disease])            

This function leverages the disease similarity information computed by Open Targets to connect Diseases to each other. 
Those links will later be used to find risk factors:

In [0]:
def connect_diseases_to_diseases(graph):
    """
    This function leverages the disease similarity information computed by Open Targets
    to connect Diseases to each other. Those links will later be used to find risk factors.
    """
    
    # Get a list of all the diseases currently in the graph.
    # We do that by looking at the objects of triples we know link to Diseases
    diseases = set([t[2] for t in graph if t[1] == 'isAboutDisease']) 
    diseases = diseases | set([t[2] for t in graph if t[1] == 'isAssociatedTo']) 
    
    # Query OpenTargets
    ot_output_diseases = {}
    ot = OpenTargetsClient()
    for disease in tqdm(diseases):
        ot_output_diseases[disease] = set()
        search_results = ot.get_similar_disease(disease)
        for search_result in search_results:
            if search_result['subject']['id'] == disease: # Safe guard
                ot_output_diseases[disease].add(search_result['object']['id'])
                
    # Turn the output we received into edges
    for src_disease, target_diseases in ot_output_diseases.items():
        for target_disease in target_diseases:
            graph.append([src_disease, 'hasGeneticClue', target_disease])

This function adds to the graph the disease classification tree:

In [0]:
def add_disease_classification(graph):
    """
    This function adds to the graph the disease classification tree.
    See, for example, https://www.targetvalidation.org/disease/EFO_0005774 .
    """

    # Get a list of all the diseases in the graph
    diseases = set([t[2] for t in graph if t[1] == 'isAboutDisease']) 
    diseases = diseases | set([t[2] for t in graph if t[1] == 'isAssociatedTo']) 
    diseases = diseases | set([t[2] for t in graph if t[1] == 'hasGeneticClue']) 

    # Query OpenTargets
    paths = set()
    ot = OpenTargetsClient()
    for disease in tqdm(diseases):
        search_results = ot.search(disease)
        if search_results != None and len(search_results) > 0:
            search_result = search_results[0]
            if search_result['id'] == disease:
                if 'efo_path_codes' in search_result['data']:
                    for path in search_result['data']['efo_path_codes']:
                        paths.add('=>'.join(path))
                        
    # Turn the output we received into edges
    for path_str in paths:
        path = path_str.split('=>')
        for index in range(0, len(path)-1):
            start = path[index]
            end = path[index+1]
            graph.append([end, 'isASpecific', start])

This function query Open Targets for the therapeutic area of all the diseases:

In [0]:
def add_disease_therapeutic_areas(graph):
    """
    This function query Open Targets for the therapeutic area of all the diseases
    """
    
    # Get a list of all the diseases in the graph
    diseases = set([t[2] for t in graph if t[1] == 'isAboutDisease']) 
    diseases = diseases | set([t[2] for t in graph if t[1] == 'isAssociatedTo'])
    diseases = diseases | set([t[2] for t in graph if t[1] == 'hasGeneticClue']) 
    diseases = diseases | set([t[2] for t in graph if t[1] == 'isASpecific']) 
    
    # Query OpenTargets
    ot_output = {}
    ot = OpenTargetsClient()
    for disease in tqdm(diseases):
        ot_output[disease] = set()
        search_results = ot.get_disease(disease)
        if search_results != None and len(search_results) > 0:
            search_result = search_results[0]
            if search_result['code'].endswith(disease) and 'therapeutic_codes' in search_result:
                for therapeutic_code in search_result['therapeutic_codes']:
                        ot_output[disease].add(therapeutic_code)
                        
    # Turn the output we received into edges
    for (disease, areas) in ot_output.items():
        for area in areas:
            graph.append([disease, 'belongsToTherapeuticArea', area])

In [0]:
def print_graph_stats(graph):
    resources = set([r[0] for r in graph]) | set([r[2] for r in graph])
    predicates = set([r[1] for r in graph])
    print ('Graph has {} edges, {} resources, {} predicates'.format(len(graph), len(resources), len(predicates)))
    display(pd.DataFrame([t for t in graph], columns=['Subject', 'Predicate', 'Object']))

We now generate the actual graph by calling the functions defined above:

In [0]:
# Step 2 => get the starting graph of paper annotations
graph = get_paper_annotations_graph(annotations)
print_graph_stats(graph)

In [0]:
# Step 3 => enrich the graph with Target - Disease links
connect_targets_and_diseases(graph)
print_graph_stats(graph)

In [0]:
# Step 4 => connect diseases to related diseases
connect_diseases_to_diseases(graph)
print_graph_stats(graph)

In [0]:
# Step 5 => add disease classification trees
add_disease_classification(graph)
print_graph_stats(graph)

In [0]:
# Step 6 => add disease therapeutic areas
add_disease_therapeutic_areas(graph)
print_graph_stats(graph)

In [0]:
# Finally, we do a last pass to remove duplicate statements
final_graph = [t.split('=>') for t in set(['=>'.join(t) for t in graph])]
print_graph_stats(final_graph)

# and we save the graph to disk
graph_df = pd.DataFrame(final_graph, columns=['subject', 'predicate', 'object'])
graph_df.to_csv('graph.csv', index=False)

### A look at some of the graph content

Let's see what the graph looks like for lung cancer (`MONDO_0008903`) and Covid-19 (`MONDO_0100096`) so far:

In [0]:
def get_neighbours(resource):
    # Extract a disease and target code=>label
    to_name = {}
    target_list = pd.read_csv('https://storage.googleapis.com/open-targets-data-releases/20.02/output/20.02_target_list.csv.gz', compression='gzip')
    for row in target_list.itertuples():
        to_name[row.ensembl_id] = row.hgnc_approved_symbol
    disease_list = pd.read_csv('https://storage.googleapis.com/open-targets-data-releases/20.02/output/20.02_disease_list.csv.gz', compression='gzip')
    for row in disease_list.itertuples():
        to_name[row.efo_id] = row.disease_full_name
    
    # Extract edges we may be interested in
    triples = [t for t in final_graph if t[0] == resource or t[2] == resource]

    # Construct a dataframe
    tmp = []
    for t in triples:
        s = '{} ({})'.format(t[0], to_name.get(t[0], '?'))
        o = '{} ({})'.format(t[2], to_name.get(t[2], '?'))
        tmp.append([s,t[1],o])
        
    return pd.DataFrame(tmp, columns=['Subject', 'Predicate', 'Object'])

In [0]:
display(get_neighbours('MONDO_0008903'))

In [0]:
display(get_neighbours('MONDO_0100096'))

# Part 3/3: Knowledge Graph Embedding Pipeline

In this section we predict risk factors for COVID-19 using graph representation learning.
Each prediction is associated a confidence score. The highest the score, the more likely such prediction is true.

Such predictions are carried out with knowledge graph embeddings, using the [AmpliGraph 1.3.1 library](https://github.com/Accenture/AmpliGraph). The knowledge grpah used is the result of step 2/3.

**This is the last step of our contribution.**

![covid19_pipeline_3.jpg](https://kagglecovid.s3-eu-west-1.amazonaws.com/covid19_pipeline_3.jpg)


### Knowledge Graph Embedding Training

We train a knowledge graph embedding model using the ComplEx model [Trouillon et al. 2016] over the knowledge grpah generated at step 2/3. ComplEx reaches state-of-the-art predictive power on agreed upon [benchmark datasets](https://docs.ampligraph.org/en/latest/experiments.html), and therefore is an obvious choice for this task.

For a comprehensive description of knowledge grpah embedding models, we suggest to read [this survey paper](https://arxiv.org/abs/2002.00388). The AmpliGraph library documention includes a [primer on the topic](https://docs.ampligraph.org/en/1.3.1/background.html).

We train for 1,000 epochs, with an embedding size `k`=200, Adam optimizer (learning rate = 1e-4), [multiclass NLL loss](https://docs.ampligraph.org/en/1.3.1/generated/ampligraph.latent_features.NLLMulticlass.html#ampligraph.latent_features.NLLMulticlass), L3 regularizer, and early stopping criteria.

Below we report a figure generated with [TensorFlow Embedding Projector](https://projector.tensorflow.org/). Each point in the cartesian space represents a bi-dimensional, t-SNE reduced representation of a 200-dimension embedding. Note that nodes and predicate types are embedded in the same space. The picture highlights the closest neighbours to COVID-19 in the uncompressed 200-dimensional space:


![covid19_embedding_space.jpg](https://kagglecovid.s3-eu-west-1.amazonaws.com/covid19_embedding_space.jpg)


### Sanity Check
To sanity-check the trained embeddings, we carve out known risk factors-disease associations for better-studied conditions (thus not including COVID-19), and we assess the predictive power of the model using agreed-upon [evaluation protocol](https://docs.ampligraph.org/en/1.3.1/generated/ampligraph.evaluation.evaluate_performance.html#ampligraph.evaluation.evaluate_performance) and [metrics](https://docs.ampligraph.org/en/1.3.1/generated/ampligraph.evaluation.mrr_score.html#ampligraph.evaluation.mrr_score). 

Such validation set includes 100 randomly selected conditions (i.e. Diseases), each connected with a number of risk factors with the `hasGeneticClue` predicate. This led to a validation set that includes 2,452 triples.

For each of those triples, we generate 7,218 corruptions, obtained with the agreed upon [protocol]((https://docs.ampligraph.org/en/1.3.1/generated/ampligraph.evaluation.evaluate_performance.html#ampligraph.evaluation.evaluate_performance)) of replacing the object of a true triple with another concept. We use 7,218 concepts (unique diseases and risk factors) to generate 7,218 corruptions for each triple in the validation set. We then score a true triple and all its corrutpions, and we report how that true triple is ranked against its 7,218 corrutpions. We repeat the same procedure for each triple in the test set, and we compute the [mean reciprocal rank (MRR, filtered settings)]((https://docs.ampligraph.org/en/1.3.1/generated/ampligraph.evaluation.mrr_score.html#ampligraph.evaluation.mrr_score)).

Our model reaches a validation mean reciprocal rank **MRR=0.07**.

The table below reports our validation results. We also include [Hits@10, Hits@100](https://docs.ampligraph.org/en/1.3.1/generated/ampligraph.evaluation.hits_at_n_score.html#ampligraph.evaluation.hits_at_n_score) values, and we compare against a [baseline that returns random predictions](https://docs.ampligraph.org/en/1.3.1/generated/ampligraph.latent_features.RandomBaseline.html#randombaseline) for positive and negative triples. 
The table shows we largely outperform the random baseline across all the metrics.

*Note MRR, Hits@10 and Hits@100 are values between [0,1], and the higher the better.*

|                  | MRR   | Hits@10 | Hits@100 |
|------------------|-------|:-------:|----------|
| Random Baseline  | 0.001 | 0.001   | 0.012    |
| Our Contribution | **0.07** | **0.137**   | **0.390**    |


As we pointed out at the top of the notebook, we had not carry out extensive hyperparametr selection yet. Such operation will likely increase the predictive power of the model, hence the quality of the embeddings used to infer risk factros for COVID-19. 

### Inference: COVID-19 Risk Factor Predictions
Once sanity-checked the model, we used the trained embeddings to predict the probability of existence of a number of hypothetical triples of the form `<COVID-19 hasGeneticClue ?>`, where `?` is a risk factor (that can be a characteristic, condition, or behaviour).

These scored hypothesis are sorted from the highest scored to the lowest. A high score indicates high likelihood that the predicted risk factor affects COVID-19.

In the cells below we describe the Knowledge Graph Embedding pipeline more in detail, and we provide the necessary source code to reproduce the experiment.

## Install the dependencies

In [0]:
! conda install tensorflow-gpu'>=1.14.0,<2.0.0' -y
! pip install ampligraph

## Import the necessary libs

In [0]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

import pandas as pd
import numpy as np
np.random.seed(117)
from ampligraph.latent_features import ComplEx, TransE, DistMult, RandomBaseline

from ampligraph.evaluation import evaluate_performance, mrr_score, hits_at_n_score, mr_score
from ampligraph.utils import save_model, restore_model


DATASET_BASE_PATH = "/kaggle/"

## Load the Knowledge graph

In [0]:
triples = pd.read_csv("graph.csv")

paper_diseases = set(triples[triples.predicate == 'isAboutDisease'].object)
paper_targets = set(triples[triples.predicate == 'isAboutTarget'].object)
new_triples = []
for row in triples.itertuples():
    if row.predicate == 'isAssociatedTo':
        if row.subject in paper_targets or row.object in paper_diseases:
            new_triples.append([row.subject, row.predicate, row.object])
    if row.predicate == 'isAboutDisease' or row.predicate == 'isAboutTarget':
        new_triples.append([row.subject, row.predicate, row.object])
    if row.predicate == 'hasGeneticClue':
        if row.subject in paper_diseases or row.object in paper_diseases:
            new_triples.append([row.subject, row.predicate, row.object])
    if row.predicate == 'isASpecific':
        new_triples.append([row.subject, row.predicate, row.object])
    if row.predicate == 'belongsToTherapeuticArea':
        new_triples.append([row.subject, row.predicate, row.object])
print (len(new_triples))


graph_df = pd.DataFrame(new_triples, columns=['subject', 'predicate', 'object'])

# this line is added for making sure that the results are reproducible.
graph_df.sort_values(by=['subject', 'predicate', 'object'], inplace=True)

graph_df.to_csv('COVID_KG_sample.csv', index=False)

graph_df.head()

print('Size of the graph:', graph_df.shape)

print(graph_df.columns)

print(graph_df.predicate.value_counts())

## Construct the training set and validation set

Our validation test consists of only the triples which have predicate `hasGeneticClue`. Hence, we can move all other triples to the training set.

In [0]:
genetic_clue_triples = graph_df[graph_df['predicate']=='hasGeneticClue']
train_set = graph_df[graph_df['predicate']!='hasGeneticClue'].values

Get all the diseases that are in the current (partial) training set - so that we don't get unseen entities - i.e. at least one triple for the test set disease should exist in training set for it to get trained.

In [0]:

disease_list =  np.unique(np.concatenate([
                    np.unique(train_set[train_set[:, 1]=='isAboutDisease'][:, 2]),
                    np.unique(train_set[train_set[:, 1]=='isAssociatedTo'][:, 2]),
                ], 0))

print('diseases in df:', len(disease_list))

From this disease list, randomly choose 100 diseases on which we will validate our model.

In [0]:
import random

np.random.seed(117)

test_set_diseases = np.random.choice(list(disease_list), 100).tolist()

#test_set_diseases = set(np.random.choice(list(disease_list), 2).tolist())
print(test_set_diseases)

Get the triples with predicate `hasGeneticClue` for these 100 diseases and form the test set. Move the other triples of that predicate to training set.

In [0]:

test_set = genetic_clue_triples[genetic_clue_triples["subject"].isin(test_set_diseases)]
train_genetic_clue_triples = genetic_clue_triples[~genetic_clue_triples["subject"].isin(test_set_diseases)]
train_set = np.concatenate([train_set, train_genetic_clue_triples], 0)
train_set = np.random.permutation(train_set)

print('Train set size:', train_set.shape)
print('Test set size:', test_set.shape)
print('Full Graph size:', graph_df.shape)


Now get a list of all the diseases (including risk factors) in our training set

In [0]:
disease_list_full =  np.unique(np.concatenate([
                        np.unique(train_set[train_set[:, 1]=='isAboutDisease'][:, 2]),
                        np.unique(train_set[train_set[:, 1]=='isAssociatedTo'][:, 2]),
                        np.unique(train_set[train_set[:, 1]=='hasGeneticClue'][:, 0]),
                        np.unique(train_set[train_set[:, 1]=='hasGeneticClue'][:, 2]),
                    ], 0))

print('diseases in df:', len(disease_list_full))

## Training 

Let's first train a [RandomBaseline model](https://docs.ampligraph.org/en/1.3.1/generated/ampligraph.latent_features.RandomBaseline.html#ampligraph.latent_features.RandomBaseline) and let's evaluate it on the test set. This model assigns random scores to the triples and their corruptions. So we expect to see a very bad performance (mrr) on the test set. 

In [ ]:
filter_triples = genetic_clue_triples.values

random_model = RandomBaseline(seed=0)

random_model.fit(train_set)

ranks = evaluate_performance(test_set.values, 
                             random_model, 
                             filter_triples=filter_triples, 
                             corrupt_side='o', 
                             entities_subset=list(disease_list_full))

print('MRR with random baseline:', mrr_score(ranks))


Now, let us train a model to perform link prediction and [calibrate it](https://arxiv.org/abs/1912.10000) to return trustworthy probability estimates:

In [0]:
filter_triples = genetic_clue_triples.values


model = ComplEx(batches_count=15, seed=0, epochs=1000, k=200, eta=20,
                optimizer='adam', optimizer_params={'lr':1e-4}, 
                verbose=True, loss='multiclass_nll',
                regularizer='LP', regularizer_params={'p':3, 'lambda':1e-3})



early_stopping = { 'x_valid': test_set.values,
                   'criteria': 'mrr', 
                  'x_filter': filter_triples, 
                  'stop_interval': 3, 
                  'burn_in': 50, 
                  'corrupt_side':'o',
                  'corruption_entities': list(disease_list_full),
                  'check_interval': 50 }

model.fit(train_set, True,early_stopping)

ranks = evaluate_performance(test_set.values, 
                             model, 
                             filter_triples=filter_triples, 
                             corrupt_side='o', 
                             entities_subset=list(disease_list_full))

print('MRR with trained ComplEx embedding model:', mrr_score(ranks))

model.calibrate(train_set, positive_base_rate=0.5, epochs=100)
save_model(model, 'output_graph.pth')

The following snippet generates the [TensorBoard Embedding Projector](https://projector.tensorflow.org/) files to visualize the embedding space (see figure at the top of the notebook). 

Import the generated files into [TensorBoard Embedding Projector](https://projector.tensorflow.org/) and follow the on-screen instructions.

[AmpliGraph documentation](https://docs.ampligraph.org/en/1.3.1/generated/ampligraph.utils.create_tensorboard_visualizations.html#ampligraph.utils.create_tensorboard_visualizations) explains how to go about in detail.

Note this step is optional, and is required only to visually inspect the embeddings.


```python
from ampligraph.utils import create_tensorboard_visualizations
create_tensorboard_visualizations(model, 'covid19_tensorboard_files')
```

## Inference: COVID-19 Risk Factor Hypothesis Predictions 

We generate our list of hypothesis for COVID-19.

Such hypothesis are triples in the form  `<COVID, hasGeneticClue, ?>`, where `?` is retrieved from the list of all the diseases (that include risk factors).

We score each hypothesis using our trained model as follows:


In [0]:
disease_id = 'MONDO_0100096' #covid-19

test_predicate = 'hasGeneticClue'

hypothesis = np.concatenate([np.array([[disease_id] * disease_list_full.shape[0]]), 
                             np.array([[test_predicate] * disease_list_full.shape[0]]),
                             disease_list_full[np.newaxis, :]],0).T
print(hypothesis.shape)

scores = model.predict_proba(hypothesis)

Get the file containing the disease code to name mappings for opentargets dataset:

In [0]:
disease_mapping_list_df = disease_list = pd.read_csv('https://storage.googleapis.com/open-targets-data-releases/20.02/output/20.02_disease_list.csv.gz', 
                                                     compression='gzip')


disease_mapping_list_df.head()


## Final results
Merge the disease codes of the hypothesis with the above list to get the disease names and display the ranked list.

**Such List contains the top-100 final results** (we clipped the list to reduce clutter in the notebook. **The complete list of predicted risk factors is saved to `predicted_covid19_risk_factors.csv`.**

**Note our results are preliminary, and we aim at refining such list in phase-2, as pointed out under the future work section at the top of this notebook.**



In [0]:
tested_hypothesis = pd.DataFrame(np.concatenate([hypothesis, 
                                                 scores[:, np.newaxis]], 1), 
                                 columns=['s','p','o','score'])

tested_hypothesis = tested_hypothesis[tested_hypothesis['o'] != disease_id]

tested_hypothesis = tested_hypothesis.sort_values(by='score', 
                                                  ascending=False)

tested_hypothesis = tested_hypothesis.merge(disease_mapping_list_df, 
                                            how='left', 
                                            left_on='o', 
                                            right_on='efo_id')[['disease_full_name', 'score']]

tested_hypothesis.columns = ['Risk Factors', 'Score']

pd.set_option('display.max_rows', 101)

tested_hypothesis.head(100)

We now save the **entire list of predictions** to disk:

In [0]:
tested_hypothesis.to_csv('predicted_covid19_risk_factors.csv')